<center><b><font size=6>Lab-5 Dimensionality Reduction<b><center>

### Some notes:

1. You can use <button>Control</button> + <button>Enter</button> to execute the current code block, and you can use <button>Shift</button> + <button>Enter</button> to execute the current block and go to the next one afterwards.

In [ ]:
1+1

2. Jupyter notebook automatically outputs the variable at the last line of a block, so you can display a pandas dataframe in a clear way instead of printing it, by putting the variable at the bottom of a block or creating a new block and putting the variable in it.

In [ ]:
df = pd.read_csv('darknet_traces.csv')
df

In [ ]:
df

In [ ]:
df.head()

In [ ]:
df.describe()

3. You are not restricted to Seaborn to make plot, and you can still use Matplotlib for your work, just choose the one that suits you the best.

In [ ]:
line = [i for i in range(100)]
plt.plot(line)

In [ ]:
sns.lineplot(line)

4. ``pd.to_datetime()`` works with nanosecond, and this is why we multiply the second by $10^9$.

In [ ]:
ts = 1623452345.3400002
pd.to_datetime(ts)

In [ ]:
pd.to_datetime(ts * 1e9)

5. All the code blocks that you have executed will affect the variables, so if you run a certain block again, the variables might be further changed.

In [ ]:
df['ts'] = df['ts'] * 1e9
df

In [ ]:
df_copy = df.copy()
df_copy['ts'] = df_copy['ts'] * 1e9
df_copy

In [ ]:
df.set_index('ts')
df

In [ ]:
df = df.set_index('ts') 
# df.set_index('ts', inplace=True)
df

### Objective: Applying following techniques
1. **Scikit-learn (sklearn)** is one of the most famous Python libraries designed for machine learning. You can use it to do literally everything for ML, such as classification and clustering. In this tutorial, besides the data scaling method, we foucs on dimensionality reduction, which reduce the number of random variables (features) to consider. Useful link: <a href="https://scikit-learn.org/stable/">official documentation</a>, <a href="https://scikit-learn.org/stable/modules/decomposition.html#decompositions">dimensionality reduction in sklearn</a>.
2. **Correlation Analysis** aims to investigate the correlations among different numerical properties. Machine Learning algorithms are sensitive to data, and if two features are highly correlated to each other, either of them can be considered redundant (repeated information), e.g., age and date of birth, if a ML model is developed based on both features, it will learn the same thing out of them, which is not only useless but also complicates the model, potentially increasing the time consumption.
3. **Principal Component Analysis (PCA)** is used to project the original dataset into a new feature space (the compoments). Simply put, PCA tries to reduce the number of variables, while preserving the characteristics and differences as much as possible. Useful link: <a href="https://en.wikipedia.org/wiki/Principal_component_analysis">Wiki</a>, <a href="https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA">PCA in sklearn</a>.

In [ ]:
# import needed python libraries

%matplotlib inline

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import random

### 1. Tutorial
Here we use IRIS dataset as an example to show how you perform data scaling and PCA.

In [ ]:
# load dataset

from sklearn import datasets
iris_data = datasets.load_iris()
columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
df_iris = pd.DataFrame(iris_data.data, columns = columns)
df_iris['type'] = 'setosa'
df_iris.loc[50:99, 'type'] = 'versicolor'
df_iris.loc[100:149, 'type'] = 'virginica'

### 1.1 Data scaling
Before performing Machine Learning tasks, data must be scaled to belong to similar numerical space (range). Sveral scalers are available, e.g., StandardScaler standardizes features by removing the mean, scaling the data to unit variance (`StandardScaler` from sklean).

In [ ]:
# make a copy of the original dataset
df_iris_copy = df_iris.copy()

# define the scaler
scaler = StandardScaler()

# for each column in the dataset, fit and transform the data
for col in columns:
    
    # fit the scaler on the data 
    scaler.fit(df_iris_copy[col].values.reshape(-1, 1))

    # transform the data
    df_iris_copy[col] = scaler.transform(df_iris_copy[col].values.reshape(-1, 1))

# OR you can do it in an easy way
# df_iris_copy[columns] = scaler.fit_transform(df_iris_copy[columns])

### 1.2 Correlation analysis
In order to analyze the correlation, we need to compute the correlation between each pair of features, which can be done using the pandas function ``.corr()`` (<a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html?highlight=corr#pandas.DataFrame.corr">documentation</a>). It calculate the Pearson correlation coefficient (<a href="https://en.wikipedia.org/wiki/Pearson_correlation_coefficient">Wiki</a>) between two numerical features, which is a value between -1 and 1. Normally, we take the absolute value, and the closer to 1 the higher the correlation.
$$ r = \frac{n(\sum xy) - (\sum x)(\sum y)}{\sqrt{[n\sum x^2 - (\sum x)^2][n\sum y^2 - (\sum y)^2]}} $$
<!-- $$ r = \frac{\sum (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum (x_i - \bar{x})^2} \sqrt{\sum (y_i - \bar{y})^2}} $$ -->
The result of correlation analysis can be displayed in a heatmap (<a href="https://seaborn.pydata.org/generated/seaborn.heatmap.html">documentation</a>), which is a symmetric matrix, indicating the correlation cocoefficient between each pair of features (even a feature with itself).

In [ ]:
# Compute the correlation matrix
correlation_matrix = df_iris_copy.drop(columns=['type']).corr().abs()

# Compute the heatmap
plt.figure(figsize=(5,4))
sns.heatmap(correlation_matrix, cmap='Blues', annot=True, vmin=.0, vmax=1, cbar_kws={'label':'Correlation'})
plt.xlabel('Feature')
plt.ylabel('Feature')
plt.title('Correlation matrix')
plt.show()

### 1.3 Principal Component Analysis (PCA)

In [ ]:
# PCA must be initialized with a random state to initialize the space
pca = PCA(random_state=15)

# .fit() is used to compute the new dimensions with number of features from 1 to the number of original features
pca.fit(df_iris_copy[columns])

### 1.4 PCA visualization
HINT: Use a number components where you meet the elbow point. i.e., if increasing the number of components does not increase much the cumulative explained variance. In the title, you can specify the percentage at the number of component you choose.

In [ ]:
# describe how much of the dataset variability is indicated by a given amount of features
explained_variance = pca.explained_variance_ratio_

# evaluate the total dataset variability while increasing the variables
cumul_exp_var = np.cumsum(explained_variance)

# percentage value to better understand the best number of components
perc_cumul_exp_var = cumul_exp_var * 100

# make the plot of cumulative explained variance wrt number of components
plt.figure(figsize=(5, 3.5))
plt.plot(perc_cumul_exp_var, marker='o')
plt.xlabel('# Principal Components (PCs)')
plt.ylabel('Cumulative explained variance [%]')
plt.xticks([i for i in range(4)], [i for i in range(1,5)])
plt.grid()
plt.title(f'3 PCs explain {round(perc_cumul_exp_var[2], 2)}% of $\sigma^2$')
plt.tight_layout()
plt.show()

### 1.5 PCA transformation

In [ ]:
# initialize the PCA with the best numbre of components, in this case, it's 3
pca = PCA(n_components=3, random_state=15)

# fit the data to new space
pca.fit(df_iris_copy[columns])

# transform the original data into PCA components
pca_result = pca.transform(df_iris_copy[columns])

# create the new dataset
df_iris_pca = pd.DataFrame(pca_result, columns=['component_1', 'component_2', 'component_3'])
df_iris_pca['type'] = df_iris['type']
df_iris_pca

### 1.6 Loading score
The loading score can be used to extract the coefficients of the linear combination of the original variables from which the principal components (PCs) are constructed. It can describe how strongly a component describes the original features and identify potentially redundant features in for a given component. Note that the score is computed as ``pca.components_.T * np.sqrt(pca.explained_variance_)``.

In [ ]:
# Compute the loading scores and create the dataframe
loadings = pd.DataFrame(
    data = pca.components_.T * np.sqrt(pca.explained_variance_), 
    columns = [f'PC{i}' for i in range(1, 4)],
    index = columns
)

plt.figure()
loadings = loadings[['PC1', 'PC2']]
loadings.sort_values(['PC1', 'PC2']).rename(columns={'PC1':'PC$_{1}$', 'PC2':'PC$_{2}$'}).plot.barh(ax=plt.gca())
plt.grid()
plt.xlabel('Loadings')
plt.ylabel('Feature')
plt.title(f'3 PCs explain {round(perc_cumul_exp_var[2], 2)}% of $\sigma^2$')
plt.show()

### 2. Exercise - Darknet Dataset
In this lab, we still focus on Darknet dataset (refer to the introduction of previous lab). Specifically, you will develop the ML pipeline to extract features from the dataset, understand the features, and perform a Principal Component Analysis (PCA).

In the ``darknet_traces.csv`` file, you will find logs of darknet traffic. Each record contains the following features:
- ts: timestamp of the received packet
- src_ip: IP address from which the packet was sent
- src_port: source port from which the packet was sent
- dst_ip: Darknet IP address that the packet reached
- dst_port: darknet port the packet reached
- proto: protocol used
- pck_len: Length of the packet in bytes
- ttl: packet time to live

### 2.1 Dataset management
1. The provided features are not suitable for our task. We need to treat it as the source data to generate a new dataset with new features calculated from available features. Specifically, you need to generate a dataset reporting the information of source IP address, and for each individual source IP address (a row), you need to get the following 16 features (Fill NaN values with 0s):
    - Number of packets
    - Number of different ports contacted
    - Number of distinct used protocols
    - Number of distinct contacted `dst_ip`
    - Min, Max, Avg, Std of the `dst_port`, treated as decimal number
    - Min, Max, Avg, Std of the `pck_len`
    - Min, Max, Avg, Std of the `ttl`
2. Use a standard scalar to scale the entire dataset.
3. Create two plots, randomly selecting a subset of the dataset (you can randomly select 1000 indices):
    - The first shows the packets sent (x-axis) and the ports (y-axis) contacted.
    - The second shows the scaled version of the packets sent and contacted ports contacted.
    - Answering the following question:
        - Are these graphs similar, why?
        - If you re-run the script (different random selection), are they still similar?
        - What is the impact of standardization?

In [ ]:
# Load the dataset
df = pd.read_csv('darknet_traces.csv')

# Add one column named 'packets', containing only 1 packet for each row
df['packets'] = 1

In [ ]:
# Your answer here

### 2.2 Correlation analysis
- For the derived dataset with new features, analyze the correlation and output the heatmap with correlation coefficient, keeping only two digits. Answering the following questions:
    - How many features have highly correlated other features?
    - How many features can you remove?
    - When one feature is correlated to more than 1 other feature, how do you decide which to remove?

In [ ]:
# Your answer here

### 2.3 PCA
Let us begin by examining dimensionality reduction and how the number of components affects PCA’s ability to describe the dataset. Note that we do not eleminate correlated features for now, but focus on the original standardized dataset.

- Fit the PCA with the scaled dataset by setting ``random_state=15``. Examine the explained variance and plot the trend of the cumulative explained variance as you increase the number of components created. How many components would you choose? And why? 

In [ ]:
# Your answer here

- From now on, keep the first 10 components, initializing PCA and fitting as well as transforming the dataset again. What is the cumulative explained variance. Is it identical to the one given in the plot of cumulative explained variance, why?

In [ ]:
# Your answer here

- Consider only the first 2 principal components (PC1 and PC2) of the 10 selected. Compute the Loading score for each of the original features with respect to the selected components. Notice that you can calculate the score as follows: <br>``scores = pca.components_.T * np.sqrt(pca.explained_variance_)``<br>
Remember that the loading scores can be seen as the weights of the each original feature used to generate the PCs. Answering the following questions:
    - Which feature contributes the most to PC1? And for PC2?
    - What are the most redundant features accordingly to the selected components?

In [ ]:
# Your answer here

### 2.4 Repeat correlation analysis after performing PCA
For the transformed data with the optimal number of components, repeat the process of correlation heatmap. Answering the following questions:
- How many features (components) can you remove now, after doing the PCA?
- What phenomena can you observe? Why?

In [ ]:
# Your answer here

### 2.5 Optional - PCA for features without standardization
Previously, we have done the whole procedure following the order from standardization to PCA, but what if we perform the PCA without standardizing the dataset? Herein, you repeat what you have done in 2.2, except that you refer to the original dataset with raw features instead of performing standardization.
1. Fit the PCA with the **original** features. Examine the explained variance and plot the trend of the cumulative explained variance as you have done before. Answering the following questions:
    - How many components would you choose now? Why?
    - Is the new result different from the previous result? Why?
2. Based on the selected number of components, re-fit the PCA on the original features.
3. Repeat the calculation and visualization of Loading score for the first 2 principal components (PC1 and PC2). Answering the following questions:
    - Which feature contributes the most to PC1? And for PC2?
    - Do you observe something abnormal? Why? How do you visualize them (tip: change the scale of reference frame)?

In [ ]:
# Your answer here

### 2.6 Optional - Correlation analysis with Spearman's rank correlation coefficient
Previously, we have done the correlation analysis using Pearson correlation coefficient, and now, you can choose to explore the Spearman correlation coefficient, by specifying the ``method`` argument in ``corr()`` function. Note that you need to use the original standardized dataset instead of the one after performing PCA. Answering the following questions:
- What differences can you observe with respect to the previous correlation analysis?
- What similarities can you observe with respect to the previous correlation analysis?
- How many features can you remove now? Is it same as before?

In [ ]:
# Your answer here